In [1]:
# Input:

filelist = ['/home/minh-doan/Leukemia_DeepLearning/LK209pres/blasts.cif',
            '/home/minh-doan/Leukemia_DeepLearning/LK209pres/normal.cif']

# Parts of file names should contain annotations of the classes, e.g. normal_file1, normal_file2, disease_1,...

In [2]:
# User's settings:

image_size = 28 # Choose the size of the bound box around objects
classes = ["normal","blast"] # classes names should be in at least 1 file name
wanted_channel = [0,2,5,6,11] # IFC typically have 12 channels indexed from 0 : 0,1,2,3....11
split = 0.8 # Set the split ratio 80% data for training, 20% for testing

In [3]:
import numpy
import tensorflow as tf
import pandas
import pickle
import os
import os.path
import skimage.io
import skimage.util.montage
import bioformats
import bioformats.formatreader
import javabridge
import math
from PIL import Image
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Increase heap space for javabridge to deal with big .cif file
javabridge.start_vm(class_path=bioformats.JARS, max_heap_size='8G')

from tensorflow.contrib.tensorboard.plugins import projector

In [4]:
# To make every bound box have the same size

def __pad_or_crop(image, image_size):
    bigger = max(image.shape[0], image.shape[1], image_size)

    pad_x = float(bigger - image.shape[0])
    pad_y = float(bigger - image.shape[1])

    pad_width_x = (int(math.floor(pad_x / 2)), int(math.ceil(pad_x / 2)))
    pad_width_y = (int(math.floor(pad_y / 2)), int(math.ceil(pad_y / 2)))
    sample = image[int(image.shape[0]/2)-4:int(image.shape[0]/2)+4, :8]

    std = numpy.std(sample)

    mean = numpy.mean(sample)

    def normal(vector, pad_width, iaxis, kwargs):
        vector[:pad_width[0]] = numpy.random.normal(mean, std, vector[:pad_width[0]].shape)
        vector[-pad_width[1]:] = numpy.random.normal(mean, std, vector[-pad_width[1]:].shape)
        return vector

    if (image_size > image.shape[0]) & (image_size > image.shape[1]):
        return numpy.pad(image, (pad_width_x, pad_width_y), normal)
    else:
        if bigger > image.shape[1]:
            temp_image = numpy.pad(image, (pad_width_y), normal)
        else:
            if bigger > image.shape[0]:
                temp_image = numpy.pad(image, (pad_width_x), normal)
            else:
                temp_image = image
        return temp_image[int((temp_image.shape[0] - image_size)/2):int((temp_image.shape[0] + image_size)/2),int((temp_image.shape[1] - image_size)/2):int((temp_image.shape[1] + image_size)/2)]

In [5]:
# Make tensor and corresponding labels

multichannel_tensors = []
labels = []

for i in range(len(filelist)):
    single_channel_tensors = []
    filename = filelist[i]
    reader = bioformats.formatreader.get_image_reader("tmp", path=filename)
    image_count = javabridge.call(reader.metadata, "getImageCount", "()I")
    channel_count = javabridge.call(reader.metadata, "getChannelCount", "(I)I", 0)
    output = filename.replace(".cif", "")

    for j in range(len(wanted_channel)):

        images = [reader.read(c=wanted_channel[j], series=image) for image in range(image_count)[::2]]

        cropped_images_this_channel = numpy.expand_dims([__pad_or_crop(image, image_size) for image in images], axis =3)

        single_channel_tensors.append(cropped_images_this_channel)


    #transform nested single_channel_tensor to multi_channel_tensor
    multichannel_tensor = numpy.concatenate((single_channel_tensors), axis = 3)

    multichannel_tensors.append(multichannel_tensor)

    label = numpy.zeros((multichannel_tensor.shape[0],len(classes)))
    for k in range(len(classes)):
        if classes[k] in filelist[i]:
            label[:multichannel_tensor.shape[0],k]=1
            labels.append(label)

T = numpy.concatenate((multichannel_tensors))
L = numpy.concatenate((labels))

In [17]:
# filename = 'T' + '.sav'
# pickle.dump(T, open(filename, 'wb'))
# print(filename, "saved !")

/home/minh-doan/Leukemia_DeepLearning/LK209pres/T.sav saved !


In [18]:
# filename = 'L' + '.sav'
# pickle.dump(L, open(filename, 'wb'))
# print(filename, "saved !")

/home/minh-doan/Leukemia_DeepLearning/LK209pres/L.sav saved !


In [6]:
# Split data set

index = numpy.arange(L.shape[0])
numpy.random.shuffle(index)

Training_index = index[:int(L.shape[0]*split)]
Training_set_temp = T[Training_index,:,:,:]
Training_labels_set_temp = L[Training_index,:]
Training_set_temp_index = numpy.arange(Training_set_temp.shape[0])

Testing_index = index[:-int(L.shape[0]*split)]
Testing_set = T[Testing_index,:,:,:]
Testing_labels_set = L[Testing_index,:]

In [7]:
# Placeholders for tensorflow

x = tf.placeholder(tf.float32, shape=[None, image_size,image_size,len(wanted_channel)]) # Last number is number of color channels
y_ = tf.placeholder(tf.float32, shape=[None, len(classes)]) # Last number is number of classes

In [8]:
# Kernels, weights, biases, settings for Deep learning

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# shape of [5, 5, len(wanted_channel), 32]. the third is the number of input channels (colors)
W_conv1 = weight_variable([5, 5, len(wanted_channel), 32])
b_conv1 = bias_variable([32])

# First convolutional layer
h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Second convolutional layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# Densely Connected Layer
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, len(classes)]) # Last number is number of classes
b_fc2 = bias_variable([len(classes)]) # Number is number of classes

h_fc2 = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_conv=tf.nn.softmax(h_fc2)

cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))

# y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [9]:
# Check for imbalance:

from collections import Counter
tuple_L = [tuple(element) for element in L]

freq = Counter(tuple_L)

import collections

size = []
for l in list(set(tuple_L)):
    print( '%s : %d' % (l, freq[l]))
    size.append(freq[l])

(0.0, 1.0) : 2920
(1.0, 0.0) : 2165


In [10]:
epoch = int(Training_labels_set_temp.shape[0]/1000)*1000

In [11]:
sess = tf.InteractiveSession()

In [12]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
finalRepresentations = []

for i in range(epoch):

    numpy.random.shuffle(Training_set_temp_index)
    batchIndex = Training_set_temp_index[0:30]

    Training_set = Training_set_temp[batchIndex,:,:,:]
    Training_labels_set = Training_labels_set_temp[batchIndex,:]

    if i % 500 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:Training_set, y_: Training_labels_set, keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
        finalRepresentations.append(h_fc2.eval(session=sess, feed_dict={x:Testing_set, keep_prob:1.0}))
        # current representation of the network. Represent how the network "see" the data.

    train_step.run(feed_dict={x: Training_set, y_: Training_labels_set, keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(session=sess,feed_dict={x: Testing_set, y_: Testing_labels_set, keep_prob: 1.0}))

step 0, training accuracy 0.3
step 500, training accuracy 0.7
step 1000, training accuracy 1
step 1500, training accuracy 1
step 2000, training accuracy 0.966667
step 2500, training accuracy 1
step 3000, training accuracy 1
step 3500, training accuracy 1
test accuracy 0.999017


In [ ]:
# If you wish to see the evolution of accuracy from early stage to late stage of DeepLearning filter
# tSNE_weights_testing_set = []

# for i in finalRepresentations:
#     tsne = TSNE(perplexity=50, n_components=2, init='pca', n_iter=5000)
#     plot_only = Testing_set.shape[0]
#     lowDWeights = tsne.fit_transform(i[0:plot_only,:])
#     tSNE_weights_testing_set.append(lowDWeights)

In [16]:
# Save metadata for Tensorboard

metadata = os.path.join('metadata.tsv')

images_tb = tf.Variable(finalRepresentations[-1], name='images')
            
def save_metadata(file):
    with open(file, 'w') as f:
        for i in range(Testing_labels_set.shape[0]):
            c = numpy.nonzero(Testing_labels_set[::1])[1:][0][i]
            f.write('{}\n'.format(c))
            
save_metadata('metadata.tsv')


with tf.Session() as sess:
    saver = tf.train.Saver([images_tb])

    sess.run(images_tb.initializer)
    saver.save(sess, os.path.join('images_tb.ckpt'))

    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = images_tb.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = metadata
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter('.'), config)

In [19]:
# Close Tensorflow session, save memory !
sess.close()

In [ ]:
# Have a careful check in this output : "projector_config.pbtxt"
# "/path/to/logdir/metadata.tsv" has to be specified, CANNOT be relative path "./metadata.tsv", nor "~/metadata.tsv"

# Type command in terminal: tensorboard --logdir="/path/to/logdir"

# Next, open web-browser, connect to http://localhost:6006.